<a href="https://colab.research.google.com/github/aainabatool/Email-Classifier/blob/main/e_mail_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **`FINE-TUNING A PRE-TRAINED LLM`**





In [ ]:
!pip install transformers datasets evaluate torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

ds = load_dataset("NotShrirang/email-spam-filter")

print(ds)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/113 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/5.40M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5171 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'label', 'text', 'label_num'],
        num_rows: 5171
    })
})


In [ ]:
from sklearn.model_selection import train_test_split

y = ds["train"]["label_num"]
X = ds["train"]["text"]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
import numpy as np

np.shape(X_train), np.shape(X_val), np.shape(X_test)

((4136,), (517,), (518,))

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
X_train[1]

"Subject: message subject\nhey i ' am julie ^ _ ^ . i just turned 18 - high school senior in houston , tx . i was waiting for such a long time until this day . i finally got my wish . . i am just simply trying to show off and meet new cyber friends . and who knows where that can lead to ; d . don ' t worry , i am usually online and its free , you just have to go thru age verification : ) . i can ' t have youngers snooping around hehe .\nhttp : / / oioaz . stuntscholar . com / ju 2 /\n"

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_dict({"text": X_train})
val_dataset = Dataset.from_dict({"text": X_val})
test_dataset = Dataset.from_dict({"text": X_test})

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],  # Ensure correct column name
        padding="max_length",
        truncation=True,
        max_length=512  # Ensures uniform length
    )

# Apply tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/4136 [00:00<?, ? examples/s]

Map:   0%|          | 0/517 [00:00<?, ? examples/s]

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

In [ ]:
tokenized_train[1]

{'text': "Subject: message subject\nhey i ' am julie ^ _ ^ . i just turned 18 - high school senior in houston , tx . i was waiting for such a long time until this day . i finally got my wish . . i am just simply trying to show off and meet new cyber friends . and who knows where that can lead to ; d . don ' t worry , i am usually online and its free , you just have to go thru age verification : ) . i can ' t have youngers snooping around hehe .\nhttp : / / oioaz . stuntscholar . com / ju 2 /\n",
 'input_ids': [101,
  3395,
  1024,
  4471,
  3395,
  4931,
  1045,
  1005,
  2572,
  7628,
  1034,
  1035,
  1034,
  1012,
  1045,
  2074,
  2357,
  2324,
  1011,
  2152,
  2082,
  3026,
  1999,
  5395,
  1010,
  19067,
  1012,
  1045,
  2001,
  3403,
  2005,
  2107,
  1037,
  2146,
  2051,
  2127,
  2023,
  2154,
  1012,
  1045,
  2633,
  2288,
  2026,
  4299,
  1012,
  1012,
  1045,
  2572,
  2074,
  3432,
  2667,
  2000,
  2265,
  2125,
  1998,
  3113,
  2047,
  16941,
  2814,
  1012,
  199

In [ ]:
from sklearn.preprocessing import LabelEncoder
import torch

# Fit LabelEncoder on the entire dataset before splitting
label_encoder1 = LabelEncoder()
label_encoder1.fit(y)  # Fit on all labels before splitting

# Transform labels
y_train_encoded = label_encoder1.transform(y_train)
y_val_encoded = label_encoder1.transform(y_val)
y_test_encoded = label_encoder1.transform(y_test)

# Convert to PyTorch tensors
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
y_val_tensor = torch.tensor(y_val_encoded, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)

# 🛠️ Remove existing 'labels' column if it already exists
if "labels" in tokenized_train.column_names:
    tokenized_train = tokenized_train.remove_columns("labels")
if "labels" in tokenized_val.column_names:
    tokenized_val = tokenized_val.remove_columns("labels")
if "labels" in tokenized_test.column_names:
    tokenized_test = tokenized_test.remove_columns("labels")

# ✅ Now, add encoded labels safely
tokenized_train = tokenized_train.add_column("labels", y_train_tensor.tolist())
tokenized_val = tokenized_val.add_column("labels", y_val_tensor.tolist())
tokenized_test = tokenized_test.add_column("labels", y_test_tensor.tolist())

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(tokenized_train, batch_size=8, shuffle=True, collate_fn=data_collator)
val_dataloader = DataLoader(tokenized_val, batch_size=8, shuffle=False, collate_fn=data_collator)
test_dataloader = DataLoader(tokenized_test, batch_size=8, shuffle=False, collate_fn=data_collator)

In [ ]:
from transformers import TrainingArguments, Trainer

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Save only when an epoch is completed
    per_device_train_batch_size=16,  # Moderate batch size to prevent OOM errors
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,  # Simulates batch size 32 without high memory usage
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,  # Less frequent logging for speed
    fp16=True,  # Mixed precision to save memory
    save_total_limit=1,  # Keep only the best model to save disk space
    report_to="none",  # Disable wandb/huggingface logging if not needed
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-18-5a59f829e5f3>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.058800,0.047283
2,0.000400,0.055859


TrainOutput(global_step=387, training_loss=0.04310554838727303, metrics={'train_runtime': 327.0379, 'train_samples_per_second': 37.941, 'train_steps_per_second': 1.183, 'total_flos': 3245737978920960.0, 'train_loss': 0.04310554838727303, 'epoch': 2.980694980694981})

In [ ]:
from evaluate import load

metric = load("accuracy")
predictions = trainer.predict(tokenized_test)
preds = predictions.predictions.argmax(-1)
accuracy = metric.compute(predictions=preds, references=y_test_encoded)

print(f"Test Accuracy: {accuracy['accuracy']:.4f}")

Test Accuracy: 0.9923


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load tokenizer and model (Replace 'your_model_path' with actual model)
model_path = "your_model_path"  # Path to your fine-tuned model
tokenizer = AutoTokenizer.from_pretrained("./results/checkpoint-387" )
model = AutoModelForSequenceClassification.from_pretrained("./results/checkpoint-387" )

# Move model to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


def classify_email(text):
    """Predicts whether the given email text is spam or ham."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Move input tensors to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get predictions
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)

    # Get class with highest probability
    predicted_class = torch.argmax(outputs.logits, dim=-1).item()

    # Label mapping (Adjust based on your dataset's labels)
    label_map = {0: "Ham", 1: "Spam"}

    return label_map[predicted_class]

In [ ]:
# Example spam email
spam_email = "Congratulations! You have won a $1000 Walmart gift card. Click here to claim your prize now"
ham_email = "Just a quick reminder about our scheduled meeting tomorrow at 10 AM. We'll be discussing the progress of the current sprint. Please review the agenda beforehand."

# Classify emails
print("Spam Email Prediction:", classify_email(spam_email))  # Expected: Spam
print("Ham Email Prediction:", classify_email(ham_email))    # Expected: Ham

Spam Email Prediction: Spam
Ham Email Prediction: Ham


In [ ]:
email_text = input("Enter youre email:")
print(f"📝 Prediction: {classify_email(email_text)}")


Enter youre email:MEET ME TODAY
📝 Prediction: Ham
